# 블로그
https://dogfoot1.tistory.com/77

# sqlite 라이브러리 임포트

In [1]:
import sqlite3

In [2]:
conn = sqlite3.connect('school.db')
cur = conn.cursor()

# 테이블 생성

In [3]:
cur.execute('''
    CREATE TABLE STUDENT(
        `학번` INTEGER PRIMARY KEY,
        `이름` TEXT,
        `교수번호` INTEGER
    );
''')

In [4]:
cur.execute('''
    CREATE TABLE PROFESSOR(
        `교수번호` INTEGER PRIMARY KEY,
        `이름` TEXT
    );
''')

# STUDENT 테이블 데이터 삽입

In [5]:
# qmark style
cur.executemany('''
    INSERT INTO STUDENT VALUES (?, ?, ?);
''', ((202401, '가나다' ,1), (202402, '라마바' ,2), (202403, '사아자' ,2)))

In [6]:
# named style
cur.executemany('''
    INSERT INTO STUDENT VALUES (:pk, :name, :fk);
''', ({'pk': 202404, 'name':'차카타' ,'fk':1}, {'pk': 202405, 'name':'파하' ,'fk':3}))

In [7]:
cur.execute('SELECT * FROM STUDENT;')

In [8]:
student = cur.fetchall()
print(student)

[(202401, '가나다', 1), (202402, '라마바', 2), (202403, '사아자', 2), (202404, '차카타', 1), (202405, '파하', 3)]


# PROFESSOR 테이블 데이터 삽입

In [9]:
pro = [(1, '교수1'), (2,'교수2'), (3,'교수3')]
cur.executemany('INSERT INTO PROFESSOR VALUES (?, ?)', pro)

In [10]:
cur.execute('SELECT * FROM PROFESSOR;')
professor = cur.fetchall()
print(professor)

[(1, '교수1'), (2, '교수2'), (3, '교수3')]


In [11]:
conn.commit()

# TABLE 보기

In [12]:
def show_table(table_name):
    cur.execute('SELECT * FROM ' + table_name)
    print(cur.fetchall())

In [13]:
show_table('STUDENT')

[(202401, '가나다', 1), (202402, '라마바', 2), (202403, '사아자', 2), (202404, '차카타', 1), (202405, '파하', 3)]


In [14]:
show_table('PROFESSOR')

[(1, '교수1'), (2, '교수2'), (3, '교수3')]


# JOIN

In [15]:
cur.execute('''
    SELECT S.학번, S.이름, P.이름 FROM STUDENT AS S, PROFESSOR AS P WHERE S.교수번호 = P.교수번호;
''')
print(cur.fetchall())

[(202401, '가나다', '교수1'), (202402, '라마바', '교수2'), (202403, '사아자', '교수2'), (202404, '차카타', '교수1'), (202405, '파하', '교수3')]


# INSERT 함수 만들기

In [16]:
# 마지막 학번 찾기
cur.execute('SELECT * FROM STUDENT ORDER BY 학번 DESC;')
a = cur.fetchone()[0]
print(a)

202405


In [17]:
# 교수 번호 찾기
cur.execute('SELECT 교수번호 FROM PROFESSOR;')
a = cur.fetchall()
print(a)

[(1,), (2,), (3,)]


In [18]:
(1,) in a

True

In [19]:
# 교수 테이블 INSERT
def add_professor(name):
    '''학번은 auto increment'''
    
    # PK auto increment를 위한 마지막 교수 번호를 얻어 온다.
    cur.execute('SELECT * FROM PROFESSOR ORDER BY 교수번호 DESC;')
    present = cur.fetchone()
    # 마지막 학번이 없으면 0으로 할당
    last_number = present[0] if present else 0
    
    # 데이터 삽입
    cur.execute('INSERT INTO PROFESSOR VALUES (?, ?)', (last_number+1, name))
    conn.commit()
    print(f'[{last_number+1}, {name}] PROFESSOR 테이블에 저장되었습니다')

In [20]:
add_professor('교수4')

[4, 교수4] PROFESSOR 테이블에 저장되었습니다


In [21]:
# 학생 테이블 INSERT
def add_student(name, p_num=None):
    '''교수번호는 auto increment'''
    
    # PK auto increment를 위해 마지막 학번을 얻어 온다
    cur.execute('SELECT * FROM STUDENT ORDER BY 학번 DESC;')
    present = cur.fetchone()
    # 마지막 학번이 없으면 202400으로 할당
    last_number = present[0] if present else 202400
    
    # 담당 교수가 배정되지 않았다면 None으로 삽입
    if p_num == None:
        cur.execute('INSERT INTO STUDENT(학번, 이름) VALUES (?, ?)', (last_number+1, name))
    # 담당 교수가 있다면
    else:
        # 교수 번호가 존재하는지 확인
        cur.execute('SELECT 교수번호 FROM PROFESSOR')
        prof = cur.fetchall()
        # 없다면
        if (p_num,) not in prof:
            print("교수 번호가 존재하지 않습니다. 다시 시도해주세요")
            return
        # 교수 번호가 존재하면 삽입
        else:
            cur.execute('INSERT INTO STUDENT VALUES (?, ?, ?)', (last_number+1, name, p_num))
    conn.commit()
    print(f'[{last_number+1} {name} {p_num}]이 STUDENT 테이블에 저장되었습니다')

In [22]:
add_student('홍길동')

[202406 홍길동 None]이 STUDENT 테이블에 저장되었습니다


In [23]:
add_student('김이박', 5)

교수 번호가 존재하지 않습니다. 다시 시도해주세요


In [25]:
show_table('STUDENT')

[(202401, '가나다', 1), (202402, '라마바', 2), (202403, '사아자', 2), (202404, '차카타', 1), (202405, '파하', 3), (202406, '홍길동', None)]


# 교수 번호 찾기

In [28]:
cur.execute('SELECT 교수번호 FROM PROFESSOR WHERE 이름 LIKE ?', ['%교수4%'])
cur.fetchone()

(4,)

In [29]:
# 교수 번호 찾기
def find_professor(p_name):
    cur.execute('SELECT 교수번호 FROM PROFESSOR WHERE 이름 LIKE ?', ['%'+p_name+'%'])
    number = cur.fetchone()
    return number[0] if number else "없음"

In [30]:
find_professor('교수1')

1

In [31]:
find_professor('교수5')

'없음'

In [32]:
add_student('김이박', find_professor('교수4'))

[202407 김이박 4]이 STUDENT 테이블에 저장되었습니다


In [33]:
add_student('김이박', find_professor('교수5'))

교수 번호가 존재하지 않습니다. 다시 시도해주세요


In [35]:
#join
cur.execute('''
    SELECT S.학번, S.이름, P.이름 FROM STUDENT AS S, PROFESSOR AS P WHERE S.교수번호 = P.교수번호;
''')
print(cur.fetchall())

[(202401, '가나다', '교수1'), (202402, '라마바', '교수2'), (202403, '사아자', '교수2'), (202404, '차카타', '교수1'), (202405, '파하', '교수3'), (202407, '김이박', '교수4')]


In [36]:
# 담당 교수가 교수2인 학생 이름만 출력
cur.execute('''
    SELECT S.이름
    FROM STUDENT AS S INNER JOIN PROFESSOR AS P
    ON S.교수번호 = P.교수번호
    WHERE P.이름 = '교수2';
''')
print(cur.fetchall())

[('라마바',), ('사아자',)]


# 담당 교수 배정

In [42]:
# 담당 교수가 NULL인 것 찾기
cur.execute('''SELECT * FROM STUDENT WHERE 교수번호 IS NULL''')
print(cur.fetchall())

[(202406, '홍길동', None)]


In [43]:
# 교수4로 UPDATE
cur.execute('''UPDATE STUDENT SET 교수번호=4 WHERE 교수번호 IS NULL''')
cur.execute('''SELECT * FROM STUDENT WHERE 교수번호 IS NULL''')
print(cur.fetchall())

[]


In [44]:
# cur.executescript('''
#     delete from STUDENT;
#     delete from PROFESSOR;
# ''')
conn.commit()

In [37]:
conn.close()

In [38]:
conn = sqlite3.connect('school.db')
cur=conn.cursor()

In [45]:
cur.execute('''
    SELECT S.학번, S.이름, P.이름 FROM STUDENT AS S, PROFESSOR AS P WHERE S.교수번호 = P.교수번호;
''')
print(cur.fetchall())

[(202401, '가나다', '교수1'), (202402, '라마바', '교수2'), (202403, '사아자', '교수2'), (202404, '차카타', '교수1'), (202405, '파하', '교수3'), (202406, '홍길동', '교수4'), (202407, '김이박', '교수4')]
